<a href="https://colab.research.google.com/github/pragneshrana/DeepLearning/blob/master/CNN/MNIST_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Instead of CIFAR10, use the MNIST data available from `torchvision.datasets.MNIST`. Download and visualise MNIST data.
* But importantly MNIST images have only 1 channel instead of 3 channels in the case of CIFAR10. Modify LeNet to work with MNIST. Train and find optimised test accuracy.

In [15]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn

## Data loading

In [16]:
import torchvision
import torchvision.transforms as transforms
trainset = torchvision.datasets.MNIST(root='./data', train=True, 
                                        download=True, 
                                        transform=transforms.ToTensor())

In [17]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True)

In [18]:
dataiter = iter(trainloader)
images, labels = dataiter.next()

print(images.shape)

print(images[1].shape)
print(labels[1].item())

torch.Size([4, 1, 28, 28])
torch.Size([1, 28, 28])
7


## Training LeNet

In [19]:
class LeNet(nn.Module):
    def __init__(self): 
        super(LeNet, self).__init__()
        self.cnn_model = nn.Sequential(
            nn.Conv2d(1, 20, 5),         # (N, 1, 28, 28) -> (N,  20, 24, 24)
            nn.ReLU(),
            nn.AvgPool2d(2, stride=2),  # (N, 20, 24, 24) -> (N,  20, 12, 12)
            nn.Conv2d(20, 40, 3),        # (N, 20, 12, 12) -> (N, 40, 10, 10)  
            nn.ReLU(),
            nn.Conv2d(40, 10, 3),        # (N, 40, 10, 10) -> (N, 10, 8, 8)  
        )
        self.fc_model = nn.Sequential(
            nn.Linear(640,400),         # (N, 640) -> (N, 400) 10*8*8 = 640
            nn.ReLU(),
            nn.Linear(400,120),         # (N, 400) -> (N, 120) 
            nn.ReLU(),
            nn.Linear(120,84),          # (N, 120) -> (N, 84)
            nn.ReLU(),
            nn.Linear(84,10)            # (N, 84)  -> (N, 10)
        )
        
    def forward(self, x):
        x = self.cnn_model(x)
        x = x.view(x.size(0), -1)
        x = self.fc_model(x)
        return x

In [20]:
batch_size = 128
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)

In [21]:
net = LeNet()

In [22]:
import torch.optim as optim

loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(net.parameters(),)


## Move to GPU

In [23]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [24]:
def evaluation(dataloader):
    total, correct = 0, 0
    for data in dataloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = net(inputs)
        _, pred = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (pred == labels).sum().item()
    return 100 * correct / total

In [25]:
net = LeNet().to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(net.parameters())

In [26]:
%%time
max_epochs = 50

for epoch in range(max_epochs):

    for i, data in enumerate(trainloader, 0):

        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        opt.zero_grad()

        outputs = net(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        opt.step()
        
    print('Epoch: %d/%d' % (epoch, max_epochs))

Epoch: 0/50
Epoch: 1/50
Epoch: 2/50
Epoch: 3/50
Epoch: 4/50
Epoch: 5/50
Epoch: 6/50
Epoch: 7/50
Epoch: 8/50
Epoch: 9/50
Epoch: 10/50
Epoch: 11/50
Epoch: 12/50
Epoch: 13/50
Epoch: 14/50
Epoch: 15/50
Epoch: 16/50
Epoch: 17/50
Epoch: 18/50
Epoch: 19/50
Epoch: 20/50
Epoch: 21/50
Epoch: 22/50
Epoch: 23/50
Epoch: 24/50
Epoch: 25/50
Epoch: 26/50
Epoch: 27/50
Epoch: 28/50
Epoch: 29/50
Epoch: 30/50
Epoch: 31/50
Epoch: 32/50
Epoch: 33/50
Epoch: 34/50
Epoch: 35/50
Epoch: 36/50
Epoch: 37/50
Epoch: 38/50
Epoch: 39/50
Epoch: 40/50
Epoch: 41/50
Epoch: 42/50
Epoch: 43/50
Epoch: 44/50
Epoch: 45/50
Epoch: 46/50
Epoch: 47/50
Epoch: 48/50
Epoch: 49/50
CPU times: user 6min 53s, sys: 1.98 s, total: 6min 55s
Wall time: 6min 57s


In [27]:
print('Test acc: %0.2f, Train acc: %0.2f' % (evaluation(testloader), evaluation(trainloader)))

Test acc: 99.27, Train acc: 99.95


**Was Just playing and got this :)**